# Assignment 5: SMILE Experiment
## Computational Methods in Psychology (and Neuroscience)
### Psychology 4500/7559 --- Fall 2020

# Objectives

Upon completion of this assignment, the student will have:

1. Used the list generation code to make experimental blocks.

2. Created a full-fledged experiment for collecting data.


# Assignment

* Write SMILE code in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A05_SMILE_Experiment_mst3k).

## Details

Your assignment is to turn the lists generated by code from the previous assignment into an experiment. As a reminder, regardless of whether you selected option 1 or option 2, this is a recognition memory experiment. This means that participants will study a list of items one at a time, and then, after a short delay, be tested for their memory of those items. In the test phase of each block, participants will see the study items again, along with an equal number of new items, and for each item they must specify whether the item is an old target item (i.e., one that was on the study list) or a new lure item. 

The high level structure of the experiment is as follows:

- Present the participant some instructions explaining the task
- Optionally provide some practice making responses
- Loop over the blocks of study--test lists

Each block of study--test lists will have the following structure:

- Wait for the participant to press a key to start the block
- Loop over the study list presenting the study items, one at a time
- Wait for a delay (we may eventually fill this with some simple math problems)
- Loop over the test list to present the test items, one at a time, waiting for a keyboard response on each item

Each study item trial will:

- Present the item for a specified duration (this should be a configuration variable at the top of your code)
- Wait an inter-stimulus duration plus some amount of jitter (these, too, should be config variables)
- Log the stimulus information, including when it appeared on the screen

Each test item trial will:

- Present the item on the screen (with either a Label or Image state) until the participant makes a keyboard response of either the key you have selected to indicate the item is "old" or the key that indicates the item is "new"
- Log the stimulus information, including when the stimulus appeared on the screen, when the participant made their response, and what response they made

It is possible to write the entire experiment in one big state machine, but it may be easier to break up these different sections into subroutines.

Be sure to refer to the class notebooks to help guide how to do all the steps above. We have some code below to help you get started.

  
* ***When you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on UVACollab.***  

In [1]:
import random
from csv import DictReader
import copy

# function to make a study/test block from the pools past in
def gen_block(pools, cond, num_items):
    # fill the study list
    study_list = []
    
    # loop over pools
    for pool in pools:
        # loop over items to add from that pool
        # this will be num_items/num_types for mixed lists
        for i in range(num_items):
            study_item = pool.pop()
            study_item.update({'novelty': 'target', 
                               'cond': cond})
            study_list.append(study_item)

    # shuffle the study_list
    random.shuffle(study_list)
    
    # copy the study list to be the start of the test list
    test_list = copy.deepcopy(study_list)
    
    # loop over pools
    for pool in pools:
        # loop over items to add from that pool
        # this will be num_items/num_types for mixed lists
        for i in range(num_items):
            test_item = pool.pop()
            test_item.update({'novelty': 'lure', 
                              'cond': cond})
            test_list.append(test_item)
    
    # shuffle the test list
    random.shuffle(test_list)
    
    return {'study': study_list, 'test': test_list}


# config variables
pos_file = 'pos_pool.csv'
neg_file = 'neg_pool.csv'
neu_file = 'neu_pool.csv'

# number of pools
num_pools = 3

# number of items in pure lists (must be evenly divisible by num_pools)
num_items_pure = 3

# number of repetitions of each block type
num_reps = 1

# verify these numbers make sense
num_items_mixed = int(num_items_pure / num_pools)
assert num_items_mixed * num_pools == num_items_pure


# load in the pools (must add in valence)
pos_pool = [dict({'valence': 'pos'}, **i) 
            for i in DictReader(open(pos_file, 'r'))]
neg_pool = [dict({'valence': 'neg'}, **i) 
            for i in DictReader(open(neg_file, 'r'))]
neu_pool = [dict({'valence': 'neu'}, **i) 
            for i in DictReader(open(neu_file, 'r'))]

# print out number of items in each pool
print('pos_pool:', len(pos_pool))
print('neg_pool:', len(neg_pool))
print('neu_pool:', len(neu_pool))

# shuffle the pools
random.shuffle(pos_pool)
random.shuffle(neg_pool)
random.shuffle(neu_pool)


# generate the blocks
blocks = []
for r in range(num_reps):
    # generate a pure pos block
    blocks.append(gen_block([pos_pool], 'pos', 
                            num_items_pure))
    
    # generate a pure neg block
    blocks.append(gen_block([neg_pool], 'neg', 
                            num_items_pure))
    
    # generate a pure neu block
    blocks.append(gen_block([neu_pool], 'neu', 
                            num_items_pure))
    
    # generate a mixed pos/neg/neu block
    blocks.append(gen_block([pos_pool, neg_pool, neu_pool], 
                            'mixed', num_items_mixed))

# shuffle the blocks
random.shuffle(blocks)

# let's see how many items we have left


blocks

pos_pool: 301
neg_pool: 292
neu_pool: 208


[{'study': [{'valence': 'neg',
    'description': 'idiot',
    'word_no': '223',
    'valence_mean': '3.1600000000000001',
    'valence_sd': '1.9099999999999999',
    'arousal_mean': '4.21',
    'arousal_sd': '2.4700000000000002',
    'dominance_mean': '3.1800000000000002',
    'dominance_sd': '2.1299999999999999',
    'word_frequency': '2',
    'novelty': 'target',
    'cond': 'mixed'},
   {'valence': 'neu',
    'description': 'mystic',
    'word_no': '891',
    'valence_mean': '6.0',
    'valence_sd': '2.21',
    'arousal_mean': '4.8399999999999999',
    'arousal_sd': '2.5699999999999998',
    'dominance_mean': '5.5199999999999996',
    'dominance_sd': '1.9299999999999999',
    'word_frequency': '3',
    'novelty': 'target',
    'cond': 'mixed'},
   {'valence': 'pos',
    'description': 'soothe',
    'word_no': '988',
    'valence_mean': '7.2999999999999998',
    'valence_sd': '1.8500000000000001',
    'arousal_mean': '4.4000000000000004',
    'arousal_sd': '3.0800000000000001',
    

In [2]:
# Load in the most common SMILE states
from smile.common import * 
from smile.scale import scale as s
from smile.startup import InputSubject

# enter configuration variables here (including the listgen variables)
font_size = 75
resp_keys = ['LEFT', 'RIGHT']
resp_map = {'lure': 'LEFT', 'target': 'RIGHT'}
ISI_dur = 0.5
ISI_jitter = 0.5
LOC_X_jitter = 200
LOC_Y_jitter = 100
inst_font_size = 50
stim_time = 2
inst_text = """
 
[u]STUDY PORTION:[/u] You will be presented with a list of study words 
    - Try to remember the presented words
    
    - You do not need to click anything in the section of the experiment
    
[u]TEST PORTION:[/u] You will then be tested with another list of words
    - Once a word is presented respond as quickly as you can with the following:
    
    - Hit the RIGHT ARROW key if you HAVE seen the word (OLD/Studied Words)
    
    - Hit the LEFT ARROW key if you HAVE NOT seen the word (NEW/ Unstudied Words)
 
 
"""


study_text = """
YOU ARE ABOUT TO START THE [u]STUDY PORTION[/u] OF THIS BLOCK

- Words will appear on the screen
- Try to remember the presented words
- Do not press anything

Press the ENTER key to\nstart. 



"""

test_text = '''
YOU ARE ABOUT TO START THE [u]TEST PORTION[/u] OF THIS BLOCK

- Words will appear on the screen
- Hit the RIGHT ARROW key if you HAVE seen the word (OLD/Studied Words)
- Hit the LEFT ARROW key if you HAVE NOT seen the word (NEW/ Unstudied Words)

Press the ENTER key to\nstart. 



'''
# listgen solution added to previous block
# create an experiment instance
exp = Experiment(name="VALENCE", fullscreen=False,show_splash=False ,resolution=(1024,768), scale_box=(1024, 768))

# YOUR CODE HERE TO BUILD THE STATE MACHINE
@Subroutine
def Instruct(self):
    # show the instructions
    with Parallel():
        top = Label(text=inst_text, font_size=inst_font_size,
              text_size=(exp.screen.width*0.75, None),halign="left",
              markup=True)
        Label(text="[u]MEMORY RECOGNITION TEST INSTRUCTIONS[/u]", font_size=font_size,
              text_size=(exp.screen.width*0.75, None),halign="center",
              markup=True, center_bottom = top.center_top)
        Label(text="Press ENTER key to continue.", font_size=inst_font_size,
              text_size=(exp.screen.width*0.75, None),halign="center",
              markup=True, center_top=top.center_bottom)
    with UntilDone():
        KeyPress(keys=['ENTER'])

        
@Subroutine
def Trial(self, block_num, trial_num, cur_trial):
    # present the stimulus
    stim = Label(text=cur_trial['description'],
                 font_size=font_size)
    with UntilDone():
        # make sure the stimulus has appeared on the screen
        Wait(until=stim.appear_time)
        
        # collect a response (with no timeout)
        kp = KeyPress(keys=resp_keys, 
                      base_time=stim.appear_time['time'],
                      correct_resp=Ref.object(resp_map)[cur_trial['novelty']])

    # log the result of the trial
    Log(name='valence_test', 
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct
       )

@Subroutine
def Study(self, block_num, trial_num, cur_trial):
    # present the stimulus
    stim = Label(text=cur_trial['description'],
                 font_size=font_size)
    with UntilDone():
        # make sure the stimulus has appeared on the screen
        Wait(until=stim.appear_time)
        Wait(1)
        
        # collect a response (with no timeout)
    
    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)
    # log the result of the trial
    Log(name='valence_study', 
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time
       )

    


# Get the subj id information
InputSubject('VALENCE')

Label(text='Welcome!\n\nPress the ENTER key to view the instructions ',
      font_size=font_size, halign='center',text_size=(exp.screen.width*0.75, None))
with UntilDone():
    KeyPress()

        
        
# show the instructions
Instruct()


Wait(0.5)
# Study(0, 0, blocks[0]['test'][0])
# Trial(0, 0, blocks[0]['test'][0])

# # loop over the blocks
with Loop(blocks) as block:
    # make sure they are ready to continue
    Label(text='Press the ENTER key to\nstart the next block.', 
          font_size=font_size, text_size=(exp.screen.width*0.75, None),halign='center')
    with UntilDone():
        KeyPress(keys=['ENTER'])
    
    
    Label(text=study_text, 
          font_size=font_size, text_size=(exp.screen.width*0.75, None),markup=True,halign="center")
    with UntilDone():
        KeyPress(keys=['ENTER'])
        
    with Loop(block.current['study']) as study:
        Study(block.i, study.i, study.current)

        
        
    # delay before the start of the test block
    Wait(ISI_dur, jitter=ISI_jitter)
    
    
    Label(text=test_text, 
          font_size=font_size, text_size=(exp.screen.width*0.75, None),markup=True,halign="center")
    with UntilDone():
        KeyPress(keys=['ENTER'])
        
    # loop over the trials
    with Loop(block.current['test']) as trial:
        Trial(block.i, trial.i, trial.current)

# make sure they are ready to continue
Label(text='You are all done!!!\nPress the ENTER key to go celebrate.', 
      font_size=font_size, halign='center',text_size=(exp.screen.width*0.75, None))
with UntilDone():
    KeyPress(keys=['ENTER'])


# run the experiment
exp.run()

[INFO   ] [Logger      ] Record log in /Users/uva/.kivy/logs/kivy_20-10-19_42.txt
[INFO   ] [Kivy        ] v1.11.1
[INFO   ] [Kivy        ] Installed at "/Users/uva/opt/anaconda3/envs/compsy/lib/python3.7/site-packages/kivy/__init__.py"
[INFO   ] [Python      ] v3.7.7 (default, May  6 2020, 04:59:01) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
[INFO   ] [Python      ] Interpreter at "/Users/uva/opt/anaconda3/envs/compsy/bin/python"
[INFO   ] [Factory     ] 184 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_imageio, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Camera      ] Provider: avfoundation
[INFO   ] [VideoGstplayer] Using Gstreamer 1.14.5.0
[INFO   ] [Video       ] Provider: gstplayer
[WARNING] [SMILE       ] Unable to import PYO!
[WARNING] [SMILE       ] Durations will be maintained, unless none are specified
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL ES 2" gra

In [7]:
from smile.log import log2dl
!ls data/VALENCE/new_test_101/20201019_173554

dl = log2dl('data/VALENCE/new_test_101/20201019_173554/log_valence_test_0.slog')
dl

import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.DataFrame(dl)
df

log_sysinfo_0.slog           state_MouseCursor_0.slog
log_valence_study_0.slog     state_Parallel_0.slog
log_valence_test_0.slog      state_ParentSet_0.slog
state_ButtonPress_0.slog     state_ProgressBar_0.slog
state_Button_0.slog          state_Rectangle_0.slog
state_Elif_0.slog            state_ResetClock_0.slog
state_Func_0.slog            state_Serial_0.slog
state_If_0.slog              state_SubroutineState_0.slog
state_Image_0.slog           state_TextInput_0.slog
state_KeyPress_0.slog        state_UpdateWidget_0.slog
state_Label_0.slog           state_Wait_0.slog
state_Loop_0.slog


,block_num,trial_num,stim_on_time,stim_on_error,resp,resp_time_time,resp_time_error,rt,correct,log_time,valence,description,word_no,valence_mean,valence_sd,arousal_mean,arousal_sd,dominance_mean,dominance_sd,word_frequency,novelty,cond,log_num
0,0,0,28.935663,0.0,LEFT,29.783628,0.000169,0.847966,True,29.783628,pos,justice,242,7.7800000000000002,1.3500000000000001,5.4699999999999998,2.54,6.4699999999999998,2.2599999999999998,114,lure,mixed,0
1,0,1,29.802498,0.0,LEFT,30.954260,0.000167,1.151762,True,30.954260,neu,pig,937,5.0700000000000003,1.97,4.2000000000000002,2.4199999999999999,5.3399999999999999,1.8799999999999999,8,lure,mixed,0
2,0,2,30.985523,0.0,RIGHT,31.515018,0.000163,0.529495,True,31.515018,neg,idiot,223,3.1600000000000001,1.9099999999999999,4.21,2.4700000000000002,3.1800000000000002,2.1299999999999999,2,target,mixed,0
3,0,3,31.552199,0.0,RIGHT,32.172708,0.000161,0.620509,True,32.172708,pos,soothe,988,7.2999999999999998,1.8500000000000001,4.4000000000000004,3.0800000000000001,5.3600000000000003,2.2400000000000002,2,target,mixed,0
4,0,4,32.202405,0.0,RIGHT,32.741109,0.000162,0.538704,True,32.741109,neu,mystic,891,6.0,2.21,4.8399999999999999,2.5699999999999998,5.5199999999999996,1.9299999999999999,3,target,mixed,0
5,0,5,32.768839,0.0,LEFT,34.058323,0.000168,1.289484,True,34.058323,neg,stench,996,2.1899999999999999,1.3700000000000001,4.3600000000000003,2.46,4.29,1.9099999999999999,1,lure,mixed,0
6,1,0,44.452269,0.0,RIGHT,45.225888,0.001507,0.773619,True,45.225888,pos,cute,97,7.6200000000000001,1.01,5.5300000000000002,2.71,4.8600000000000003,2.3199999999999998,5,target,pos,0
7,1,1,45.252251,0.0,RIGHT,45.885363,0.000162,0.633113,True,45.885363,pos,wink,1033,6.9299999999999997,1.8300000000000001,5.4400000000000004,2.6800000000000002,5.7000000000000002,1.77,7,target,pos,0
8,1,2,45.902122,0.0,LEFT,46.911403,0.000160,1.009281,True,46.911403,pos,rescue,352,7.7000000000000002,1.24,6.5300000000000002,2.5600000000000001,6.4500000000000002,2.29,15,lure,pos,0
9,1,3,46.935454,0.0,RIGHT,47.678496,0.000196,0.743042,True,47.678496,pos,snow,575,7.0800000000000001,1.8300000000000001,5.75,2.4700000000000002,5.7999999999999998,1.97,59,target,pos,0
